# Creating an IPO Spider

## Table of Contents

1. Introduction
2. Install & Import Packages
3. Access HTML and Create Selector Object
4. Create & Run IPO Spider
5. Create & Style IPO Dataframe

## 1. Introduction

350+ companies and counting have IPO'd on U.S. exchanges in 2020, the highest number since 2000, including big names like Palantir, Asana, Snowflake, and McAfee. Others like Airbnb, Doordash, Instacart, and Robinhood are in the bullpen. It would be great to see what these companies are all about, learn their key products, and to have that info in one place.

To source info, let's look at the public website https://stockanalysis.com/ipos/2020-list/. We'll build a simple web spider that will scrape the links to each company on the site, follow each company link, scrape each company's description, and output a dictionary with keys as company names & tickers and values as company descriptions. We'll then convert the dictionary that the spider generated into a styled dataframe. We'll create the web spider using scrapy and use both xpath and css methods to access html elements. The heart of the spider are its two parsing methods using css to access html elements - the first to extract links that the spider will follow and the second to scrape the company name & ticker and company description from the new site for each company. Constructing the parsing methods requires time upfront to inspect the site's html in order to get the right css selector for the target html elements. The purpose is to demonstrate how to build a web spider and to create a single table of high-level company descriptions for prospective investors. Future potential areas of scraping, for anyone interested, includes financial detail, including revenue, EBITDA, and free cash flow. 

## 2. Install & Import Packages

In [1]:
import pandas as pd

!pip install scrapy
import scrapy
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
import requests

## 3. Access HTML and Create Selector Object

In [2]:
# Url containing html
url = "https://stockanalysis.com/ipos/2020-list/"

# Get html source code using requests.get and .content and store in string html
html = requests.get(url).content

# Create the Selector object sel from html. Remember Selector returns a list
sel = Selector(text = html)

# Check number of html elements. We use xpath here, // means all generations and * is a wildcard for any child elements
print("Number of elements in html document: ", len(sel.xpath('//*')))

Number of elements in html document:  3059


## 4. Create & Run IPO Spider

In [8]:
# Create Spider class
class IPO_Spider(scrapy.Spider):
  name = "ipo_spider"
  # start_requests method
  def start_requests(self):
    yield scrapy.Request(url = url,
                         callback = self.parse_front)
  # 1st parsing method using css - link to follow
  def parse_front(self, response):
    links = response.css('table.maintable td a::attr(href)').extract() # extract href attribute from all chldren a of all children td of table with class maintable 
    for url in links:
      yield response.follow(url = url,
                            callback = self.parse_pages)
  # 2nd parsing method using css - company name & ticker, description
  def parse_pages(self, response):
    name_ticker = response.css('h1::text').extract_first().strip() # extract and strip text from h1
    description = response.css('p.desc::text').extract_first().strip() # extract and strip text from child p with class desc
    ipo_dict[name_ticker] = description # create dictionary with key name_ticker and value description

# Initialize dictionary outside of Spider class
ipo_dict = dict()

# Run Spider
process = CrawlerProcess()
process.crawl(IPO_Spider)
process.start()

2020-10-29 12:39:43 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: scrapybot)
2020-10-29 12:39:43 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.9 (default, Aug 31 2020, 12:42:55) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 2.9.2, Platform Linux-4.15.0-118-generic-x86_64-with-redhat-8.2-Ootpa
2020-10-29 12:39:43 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-10-29 12:39:44 [scrapy.crawler] INFO: Overridden settings:
{}
2020-10-29 12:39:44 [scrapy.extensions.telnet] INFO: Telnet Password: ee3d05d6906d1b89
2020-10-29 12:39:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-10-29 12:39:44 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.

In [9]:
# Check the spider's dictionary output - as we wanted, the keys are company & ticker and the values are company descriptions
ipo_dict

{'Absolute Software Corporation (ABST)': 'Absolute Software delivers a cloud-based service that supports the management and security of computing devices, applications, and data for a variety of organizations globally. Our differentiated technology is rooted in our patented Persistence® technology, which is embedded in the firmware of laptop, desktop, and tablet devices by almost every major global computer manufacturer. Enabling a permanent digital tether between the endpoint and the organization that distributed it, we provide IT and security personnel with connectivity, visibility, and control, whether a device is on or off the corporate network, and empower them with Self-Healing Endpoint® security to ensure mission critical applications remain healthy and deliver intended value. Our technology is embedded in over a half-billion devices and we currently serve more than 13,000 commercial customers with over 10.8 million activated licenses globally. Our solutions are delivered in a s

## 5. Create & Style IPO Dataframe 

In [13]:
# Create dataframe ipo from dictionary using pd.DataFrame.from_dict
ipo = pd.DataFrame.from_dict(ipo_dict, orient='index', columns=['Description'])

# Dataframe has alot of companies and text so let's set row and column display options 
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Left justify the text using ipo.style.set_properties. Left justify the column name using set_table_styles([dict(selector='th', props=[('text-align', 'left')
ipo.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

# Since there are many rows, to display the whole dataframe without vertical scroll bar, click on Cell --> Cell Outputs --> Toggle Scrolling

,Description
Absolute Software Corporation (ABST),"Absolute Software delivers a cloud-based service that supports the management and security of computing devices, applications, and data for a variety of organizations globally. Our differentiated technology is rooted in our patented Persistence® technology, which is embedded in the firmware of laptop, desktop, and tablet devices by almost every major global computer manufacturer. Enabling a permanent digital tether between the endpoint and the organization that distributed it, we provide IT and security personnel with connectivity, visibility, and control, whether a device is on or off the corporate network, and empower them with Self-Healing Endpoint® security to ensure mission critical applications remain healthy and deliver intended value. Our technology is embedded in over a half-billion devices and we currently serve more than 13,000 commercial customers with over 10.8 million activated licenses globally. Our solutions are delivered in a software-as-a-service (“SaaS”) model, where customers access our service through the cloud-based Absolute service. Our solutions are offered in specific versions for the (i) enterprise and government, and (ii) education verticals. All versions are available in three editions: Visibility, Control, and Resilience, each of which provides a different subset of product features and functionality. We also offer a Home and Office edition of our service, which is targeted to consumers and home office professionals."
"Praxis Precision Medicines, Inc. (PRAX)","Praxis Precision Medicines, a clinical-stage biopharmaceutical company, develops therapies for central nervous system disorders characterized by neuronal imbalance. Its lead product candidates include PRAX-114, an extrasynaptic-preferring GABAA receptor positive allosteric modulator that is in Phase IIa clinical trial for the treatment of major depressive disorder and perimenopausal depression; and PRAX-944, a selective small molecule inhibitor of T-type calcium channels, which is in Phase IIa clinical trial for the treatment of essential tremor. The company is also developing PRAX-562, a persistent sodium current blocker that is in Phase I clinical trial to treat severe pediatric epilepsy and adult cephalgia; PRAX-222, an antisense oligonucleotide for patients with gain-of-function (GOF) SCN2A epilepsy; and KCNT1 program for the treatment of KCNT1 GOF epilepsy. It has a cooperation and license agreement with RogCon Inc.; a license agreement Purdue Neuroscience Company; and a research collaboration, option, and license agreement with Ionis Pharmaceuticals, Inc. The company was incorporated in 2015 and is based in Cambridge, Massachusetts."
"Eastern Bankshares, Inc. (EBC)","Eastern Bankshares provides commercial banking products and services primarily to retail, commercial, and small business customers. The company offers interest-bearing and non interest-bearing checking deposits, money market deposits, savings deposits, and certificates of deposits. It also offers commercial and industrial loans, commercial real estate and construction loans, business banking loans, residential real estate loans, and home equity and other consumer loans. Its personal banking products and services also include debit and credit cards; mortgage and personal loans; personal and cash reserve lines of credit; auto and student loans; retirement planning products and services; and online learning services in the areas of finance. The company's business banking products and services also include preferred term loans, small business administration loans, lines of credit, cash reserves, cash management, merchant services, escrow express service, correspondent and government banking, international banking, interest on lawyers trust accounts services, products and services for not-for-profit and healthcare, and business telephone banking. In addition, it offers trust and investment products and services; community develo

In [12]:
# Checking the shape, we have 353 IPOs in 2020 
ipo.shape

(353, 1)